# topic modeling playground
In this notebook, different concepts for topic modeling will be tested and evaluated.

In [99]:
import re

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# preprocessing
import nltk
nltk.download('stopwords')
import spacy

# Plotting
import pyLDAvis
import pyLDAvis.gensim_models

import numpy as np

from matplotlib import pyplot

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/supelir/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Structure

In [100]:
from enum import Enum


class Party(Enum):
    AFD = 0
    CDU = 1
    FDP = 2
    GRUENE = 3
    LINKE = 4
    SPD = 5



## Preprocessing

In [101]:
nlp = spacy.load('de_core_news_md')
from nltk.corpus import stopwords
from spacy.lang.de.stop_words import STOP_WORDS

# stopwords
nltk_stopwords = stopwords.words('german')

# build stopwords list
all_stopwords = list(set(STOP_WORDS) | set(nltk_stopwords))
with open('custom_stopwords.txt', 'r', encoding='utf-8') as f:
    all_stopwords += [line.strip() for line in f.readlines()]

# Load files
party_text = {}
for party in Party:
    all_stopwords.extend(['{}'.format(party.name.lower())])
    with open('resources/' + party.name + '.txt', encoding='utf-8', errors='ignore') as txt:
        file = " ".join(l for l in txt)
        # remove gender *
        file = re.sub(r'\*innen(\w*)\s', r'\1 ', file)
    party_text[party] = file



In [102]:

sections = re.split(r'\n\s*\n', party_text[Party.FDP])

partie_wordbags_mod = []


for section in sections:

    partie_wordbag = gensim.utils.simple_preprocess(section)

    #filter stopwords
    partie_wordbag_spacy = [word for word in partie_wordbag if word not in all_stopwords]

    partie_wordbags_mod.append(partie_wordbag_spacy)


# make bigrams
bigram = gensim.models.Phrases(partie_wordbags_mod, min_count=5, threshold=100)
# make trigrams
trigram = gensim.models.Phrases(bigram[partie_wordbags_mod], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags and token.lemma_ not in all_stopwords])
    return texts_out

words_trigrams = make_trigrams(partie_wordbags_mod)
words_lematized = lemmatization(words_trigrams)

id2word = corpora.Dictionary(words_lematized)
texts = words_lematized
corpus = [id2word.doc2bow(text) for text in texts]




In [103]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                             id2word=id2word,
                                             num_topics=10,
                                             )

lda_model.print_topics(num_topics=10, num_words=10)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=words_lematized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.610602864799805

Coherence Score:  0.27204101357481975


## V

In [104]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis





/home/supelir/.local/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.029839 -0.042069       1        1  18.916152
6     -0.008414  0.016266       2        1  12.442573
3     -0.023651 -0.012092       3        1  11.764097
7      0.055266  0.013049       4        1  10.978269
9      0.010477  0.032822       5        1  10.349080
5      0.015481 -0.011590       6        1   9.517628
2     -0.026880 -0.042576       7        1   7.217206
4     -0.017612  0.025750       8        1   7.097792
0     -0.045702  0.012289       9        1   6.047668
1      0.011196  0.008151      10        1   5.669535, topic_info=                 Term       Freq      Total Category  logprob  loglift
1983          pflegen  12.000000  12.000000  Default  30.0000  30.0000
1400               co  17.000000  17.000000  Default  29.0000  29.0000
16            bildung  24.000000  24.000000  Default  28.0000  28.0000
1092        beruflich  13.000000  13.000000  Default  27.0000  27.0000
126      wissenschaft   8.000000   8.000000  Default  26.0000  26.0000
...               ...        ...        ...      ...      ...      ...
375          digitale   1.235052  29.019386  Topic10  -6.2936  -0.2868
70             modern   1.206859  26.922542  Topic10  -6.3167  -0.2349
148   digitalisierung   1.130790  20.429030  Topic10  -6.3818  -0.0240
320            privat   1.171026  25.145829  Topic10  -6.3468  -0.1967
44           freiheit   1.100151  26.036178  Topic10  -6.4092  -0.2940

[729 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
256       1  0.200050       abbauen
256       2  0.100025       abbauen
256       3  0.100025       abbauen
256       4  0.100025       abbauen
256       5  0.100025       abbauen
...     ...       ...           ...
4491      2  0.688419    übermorgen
530       1  0.572186    überprüfen
255       2  0.366892   überzeugung
255       7  0.366892   überzeugung
3062      2  0.467659  übungsleiter

[1554 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 7, 4, 8, 10, 6, 3, 5, 1, 2])